<a href="https://colab.research.google.com/github/krs-lab/codon-optimization/blob/main/KRS_codon_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Load packages and dependencies
### This script generates codon optimized sequences for protein expression
### The script takes:
###   - a FASTA formatted protein sequences
###   - an optomized codon table for the organism of choice
### The output is:
###   - a FASTA formatted file with multiple codon optimized versions of the input sequence

#############################
### Here we simply import various packages

### Pandas and Numpy handle data
import pandas as pd
import numpy as np
import random

### ipywidgets handles fancy input and output
from ipywidgets import widgets, interact, interactive

### matplotlib for plotting
%matplotlib inline
import matplotlib.pyplot as plt

### scipy for ttest function
from scipy import stats

### re for regex functions
import re

import seaborn as sns
sns.set_theme()

!pwd
!ls

/content
drive  sample_data  Test_output.fas


In [ ]:
# @title Which codon table should we use?
### This defines a "radiobutton"
### which allows us to select an
### optimized codon table
radiobutton = widgets.RadioButtons(
    options=['E. coli', 'M. smegmatis'],
    value='E. coli', # Default
    description='Table:',
    disabled=False
)

### E. coli is the default codon table used
##codon_table_file_name = "codon_table_coli_optimized.txt"

### Captures input and set codon table
### Codon tables are hardcoded
def set_choice(x):
  global codon_table_file_name
  if(x.new == "E. coli"):
    codon_table_file_name = "codon_table_coli_optimized.txt"
  if(x.new == "M. smegmatis"):
    codon_table_file_name = "codon_table_myco_optimized_for_TWIST.txt"
  return x.new

### Event handlers...
radiobutton.observe(set_choice,names='value')

### This shows the dropdowns
display(radiobutton)


RadioButtons(description='Table:', options=('E. coli', 'M. smegmatis'), value='E. coli')

In [ ]:
# @title Import the appropriate codon table...
### Here we read in the codon table...
### These were originally read in from my Google Drive, but there are access issues with third party users, so I'm just hardcoding them.
# codon_table_filehandle = open("/content/drive/My Drive/Perl/Gene_Synthesizer/"+codon_table_file_name, 'r')

codon_table = []
if(codon_table_file_name == "codon_table_coli_optimized.txt"):
  codon_table = [['a', ['gcg', '0.36'], ['gca', '0.21'], ['gct', '0.16'], ['gcc', '0.27']], ['r', ['cgt', '0.49'], ['cgc', '0.51']], ['d', ['gat', '0.63'], ['gac', '0.37']], ['n', ['aat', '0.45'], ['aac', '0.55']], ['c', ['tgt', '0.45'], ['tgc', '0.55']], ['g', ['ggg', '0.15'], ['gga', '0.11'], ['ggt', '0.34'], ['ggc', '0.40']], ['e', ['gag', '0.31'], ['gaa', '0.69']], ['q', ['cag', '0.65'], ['caa', '0.35']], ['h', ['cat', '0.57'], ['cac', '0.43']], ['i', ['att', '0.55'], ['atc', '0.45']], ['k', ['aag', '0.23'], ['aaa', '0.77']], ['l', ['ttg', '0.14'], ['tta', '0.14'], ['ctg', '0.52'], ['ctt', '0.10'], ['ctc', '0.10']], ['m', ['atg', '1.00']], ['f', ['ttt', '0.57'], ['ttc', '0.43']], ['p', ['ccg', '0.53'], ['cca', '0.19'], ['cct', '0.16'], ['ccc', '0.12']], ['s', ['agt', '0.15'], ['agc', '0.28'], ['tcg', '0.15'], ['tca', '0.12'], ['tct', '0.15'], ['tcc', '0.15']], ['t', ['acg', '0.27'], ['aca', '0.12'], ['act', '0.17'], ['acc', '0.44']], ['y', ['tat', '0.57'], ['tac', '0.43']], ['w', ['tgg', '1.00']], ['v', ['gtg', '0.37'], ['gta', '0.15'], ['gtt', '0.26'], ['gtc', '0.22']]]
elif(codon_table_file_name == "codon_table_myco_optimized_for_TWIST.txt"):
      codon_table = [['a', ['gcc', '0.52'], ['gcg', '0.48']], ['r', ['cgt', '0.35'], ['cgc', '0.45'], ['cgg', '0.20']], ['d', ['gat', '0.50'], ['gac', '0.50']], ['n', ['aat', '0.31'], ['aac', '0.69']], ['c', ['tgt', '0.38'], ['tgc', '0.62']], ['g', ['ggt', '0.29'], ['ggc', '0.34'], ['gga', '0.20'], ['ggg', '0.17']], ['e', ['gaa', '0.49'], ['gag', '0.51']], ['q', ['caa', '0.24'], ['cag', '0.76']], ['h', ['cat', '0.28'], ['cac', '0.72']], ['i', ['atc', '1.00']], ['k', ['aaa', '0.38'], ['aag', '0.62']], ['l', ['ctc', '0.36'], ['ctg', '0.64']], ['m', ['atg', '1.00']], ['p', ['ccc', '0.39'], ['ccg', '0.61']], ['f', ['ttc', '1.00']], ['s', ['tcc', '0.25'], ['tcg', '0.46'], ['agc', '0.29']], ['t', ['acc', '0.65'], ['acg', '0.35']], ['v', ['gtc', '0.47'], ['gtg', '0.53']], ['w', ['tgg', '1.00']], ['y', ['tat', '0.35'], ['tac', '0.65']]]
else:
    print("ERROR")

#for line in codon_table_filehandle:
#  line = line.lower().rstrip()
#  #print(line)
#  #print("match_test:", re.fullmatch('^[acdefghiklmnpqrstvwy]\s*$',line))
#  if re.fullmatch('^[acdefghiklmnpqrstvwy]\s*$',line):
#    #print("in if, line is ",line)
#    ## new column in array
#    codon_table.append([line])
#  else:
#    match_result = re.fullmatch("^([autgc]{3}) (\d.\d{2})$", line)
#    if match_result:
#      codon = match_result.group(1)
#      codon = re.sub("u","t",codon)
#      freq = match_result.group(2)
#      ## push this into codon and freq into element of array
#      (codon_table[-1]).append([codon,freq])

print(codon_table)


[['a', ['gcg', '0.36'], ['gca', '0.21'], ['gct', '0.16'], ['gcc', '0.27']], ['r', ['cgt', '0.49'], ['cgc', '0.51']], ['d', ['gat', '0.63'], ['gac', '0.37']], ['n', ['aat', '0.45'], ['aac', '0.55']], ['c', ['tgt', '0.45'], ['tgc', '0.55']], ['g', ['ggg', '0.15'], ['gga', '0.11'], ['ggt', '0.34'], ['ggc', '0.40']], ['e', ['gag', '0.31'], ['gaa', '0.69']], ['q', ['cag', '0.65'], ['caa', '0.35']], ['h', ['cat', '0.57'], ['cac', '0.43']], ['i', ['att', '0.55'], ['atc', '0.45']], ['k', ['aag', '0.23'], ['aaa', '0.77']], ['l', ['ttg', '0.14'], ['tta', '0.14'], ['ctg', '0.52'], ['ctt', '0.10'], ['ctc', '0.10']], ['m', ['atg', '1.00']], ['f', ['ttt', '0.57'], ['ttc', '0.43']], ['p', ['ccg', '0.53'], ['cca', '0.19'], ['cct', '0.16'], ['ccc', '0.12']], ['s', ['agt', '0.15'], ['agc', '0.28'], ['tcg', '0.15'], ['tca', '0.12'], ['tct', '0.15'], ['tcc', '0.15']], ['t', ['acg', '0.27'], ['aca', '0.12'], ['act', '0.17'], ['acc', '0.44']], ['y', ['tat', '0.57'], ['tac', '0.43']], ['w', ['tgg', '1.00']],

In [ ]:
# @title Enter the protein sequence we're going to codon optimize.
### Now we create a prompt to copy in the sequence in a textarea box

fasta_sequence = widgets.Textarea(
    value='',
    layout= widgets.Layout(width='75%'),
    placeholder='Type something',
    description='String:',
    disabled=False
)

fasta_raw_text = ""

def on_change(text):
  fasta_raw_text = text.new
  #print(fasta_raw_text)
  return text.new

### Event handlers...
fasta_sequence.observe(on_change,names='value')

### This shows the text field
display(fasta_sequence)

### The upshot of this is that whever you pasted in will be recorded as "fasta_raw_text"
### presumably in FASTA format
### but, regardless, we'll strip the header line if present


Textarea(value='', description='String:', layout=Layout(width='75%'), placeholder='Type something')

In [ ]:
# @title Digest protein sequence and generate 10 codon optimized DNA sequences.
fasta_raw_text = fasta_sequence.value

### digest the input sequence
### ignore a first line starting with a >
### otherwise add to growing sequence
protein_sequence = ""

for seq_line in fasta_raw_text.splitlines():
  if re.fullmatch("^>.+$",seq_line):
    pass
  else:
    protein_sequence = protein_sequence+seq_line.lower().rstrip()

print("Digested protein sequence is:",protein_sequence)

gene_sequences = [""]*10
aa = ""
num_codons = 0
rand_num = 0

for x in range(len(gene_sequences)):
  for i in range(len(protein_sequence)):
    aa = protein_sequence[i]
    ## find this aa in the codon table
    for row in codon_table:
      if(aa == row[0]):
        ## Ok! We found our codon
        ## How many codons does it have?
        num_codons = (len(row) - 1)
        #print("For",aa,"we found this many codons:",num_codons)
        ## generate a random number between 0 and 1
        rand_num = random.random()
        moving_max = 0
        ## the bit below was used to double check that this works
        #for j in range(num_codons):
        #  print(row[j+1][0],row[j+1][1], end =" ")
        #print("")
        for j in range(num_codons):
          moving_min = moving_max
          moving_max = moving_max + float(row[j+1][1])
          if((rand_num > moving_min) and (rand_num < moving_max)):
            #print("For",aa,"we chose",row[j+1][0],", given rand of",rand_num)
            gene_sequences[x] = gene_sequences[x]+row[j+1][0]

for x in range(len(gene_sequences)):
  print("Final gene sequence",x,gene_sequences[x])


Digested protein sequence is: mghtlyapggydimgylrqirnrpnpqvelgpvdlscalvlcdlkqkdtpvvyaseaflemtgynrhevlgrncrflqspdgmvkpkstrkyvdsntiytmkkaidrnaevqvevvnfkkngqrfvnfltmipvrdetgeyrysigfqcete
Final gene sequence 0 atgggccataccctgtacgcaccaggtggatacgacattatggggtatctgcgccagatccgcaaccgcccgaacccccaggttgagctgggtcctgtggatctcagttgcgcgttggtattgtgtgatctgaaacagaaagataccccagttgtgtatgcctccgaagcgttcctcgaaatgacgggttacaatcgccatgaagttctgggccgtaattgccgtttcctccagagtccggacgggatggtgaaaccgaaaagtacccgtaagtacgttgattctaacaccatttacaccatgaaaaaagcgattgaccgtaatgcagaggttcaagtggaagttgtcaattttaaaaagaacggccagcgcttcgtcaattttctgactatgatccccgtgcgcgacgaaactggtgagtatcgctactctatcggctttcagtgcgaaactgaa
Final gene sequence 1 atgggccacaccctttatgcacccgggggatacgacatcatgggttatttacgccagattcgtaaccgcccaaacccacaggtagaactggggccggtggacttaagttgtgcattagtgttgtgcgacttaaaacagaaagacacgccagtggtgtatgcctccgaagccttcttggagatgaccgggtataaccgccatgaagtgctgggtcgtaattgtcgttttctgcagtccccggatggtatggttaaaccaaaaagcacccgcaagtacgttgattccaacaccatctataccatgaaaaaagctatcg

In [ ]:
# @title Set filename for output.
### Lets get a filename from the user for the output...

print("Please provide a filename for the output FASTA file:")

filename_box = widgets.Text(
    value='',
    layout= widgets.Layout(width='50%'),
    placeholder='Type something',
    description='Filename:',
    disabled=False
)

output_filename = ""

def on_change(text):
  output_filename = text.new
  #print(fasta_raw_text)
  return text.new

### Event handlers...
filename_box.observe(on_change,names='value')

### This shows the text field
display(filename_box)

### The upshot of this is that whever you pasted in will be recorded as "fasta_raw_text"
### presumably in FASTA format
### but, regardless, we'll strip the header line if present

Please provide a filename for the output FASTA file:


Text(value='', description='Filename:', layout=Layout(width='50%'), placeholder='Type something')

In [1]:
# @title Download output.
### Create output file
with open(filename_box.value, mode='w') as fasta_file_object:
  for x in range(len(gene_sequences)):
    print(">sequence_",x,"\n",gene_sequences[x],file = fasta_file_object)

fasta_file_object.close()

print("Expand the File folder menu to the left and you will find the output fasta file.")

NameError: name 'filename_box' is not defined